In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0].split('BecaNLP')[0],'BecaNLP/Utils'))

import NLPUtils as nlp
from NLPUtils.datasets import imdb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

In [ ]:
from NLPUtils.datasets.utils import NgramTextVectorizer
from sklearn.naive_bayes import MultinomialNB

train_corpus = imdb.train_reader()
indices = nlp.datasets.utils.split_dev_kfolds(len(train_corpus),dev_size=.2,random_state=1273)
train_idx, dev_idx = indices[0]
dev_corpus, train_corpus = train_corpus.iloc[dev_idx,:], train_corpus.iloc[train_idx,:]

vocabulary = None#imdb.vocab_file_reader()
unk_token = 'UNK'
ngram_range = (1,1)
token_pattern = r'\b\w+\b'
min_freq, max_freq = 0, np.inf
max_features = 10000
vectorizer = NgramTextVectorizer(vocabulary=vocabulary,ngram_range=ngram_range,
                min_freq=min_freq,max_freq=max_freq,unk_token=unk_token,max_features=max_features)
vectorizer.fit(train_corpus['comment'])

X_train = vectorizer.transform(train_corpus['comment'])
X_dev = vectorizer.transform(dev_corpus['comment'])
print(X_train.shape)
print(X_dev.shape)

In [6]:
y_train = train_corpus['rate'].values.copy()
y_train[y_train < 5] = 0
y_train[y_train > 6] = 1
y_dev = dev_corpus['rate'].values.copy()
y_dev[y_dev < 5] = 0
y_dev[y_dev > 6] = 1

print('Binary classification:')
print('----------------------')
print('Number of zeros in train: ',(y_train == 0).sum())
print('Number of ones in train: ',(y_train == 1).sum())
print('Number of zeros in dev: ',(y_dev == 0).sum())
print('Number of ones in dev: ',(y_dev == 1).sum())

classifier1 = MultinomialNB()
classifier1.fit(X_train,y_train)
y_predict = classifier1.predict(X_dev)
n_correct = (y_predict == y_dev).sum()
n_total = y_predict.size
print('Number of correctly predicted samples: {}/{} ({:.2f}%)'.format(n_correct,n_total,n_correct/n_total*100))

Binary classification:
----------------------
Number of zeros in train:  9965
Number of ones in train:  10035
Number of zeros in dev:  2535
Number of ones in dev:  2465
Number of correctly predicted samples: 4256/5000 (85.12%)


In [7]:
y_train = train_corpus['rate'].values.copy()
y_train[(y_train == 1) * (y_train == 2)] = 0
y_train[(y_train == 3) * (y_train == 4)] = 1
y_train[(y_train == 7) * (y_train == 8)] = 2
y_train[(y_train == 9) * (y_train == 10)] = 3


y_dev = dev_corpus['rate'].values.copy()
y_dev[(y_dev == 1) * (y_dev == 2)] = 0
y_dev[(y_dev == 3) * (y_dev == 4)] = 1
y_dev[(y_dev == 7) * (y_dev == 8)] = 2
y_dev[(y_dev == 9) * (y_dev == 10)] = 3

print('4-category classification:')
print('--------------------------')
print('N 0 train: ',(y_train == 0).sum())
print('N 1 train: ',(y_train == 1).sum())
print('N 2 train: ',(y_train == 2).sum())
print('N 3 train: ',(y_train == 3).sum())

print('N 0 dev: ',(y_dev == 0).sum())
print('N 1 dev: ',(y_dev == 1).sum())
print('N 2 dev: ',(y_dev == 2).sum())
print('N 3 dev: ',(y_dev == 3).sum())

classifier1 = MultinomialNB()
classifier1.fit(X_train,y_train)
y_predict = classifier1.predict(X_dev)
n_correct = (y_predict == y_dev).sum()
n_total = y_predict.size
print('Number of correctly predicted samples: {}/{} ({:.2f}%)'.format(n_correct,n_total,n_correct/n_total*100))

4-category classification:
--------------------------
N 0 train:  0
N 1 train:  4089
N 2 train:  1791
N 3 train:  1923
N 0 dev:  0
N 1 dev:  1011
N 2 dev:  493
N 3 dev:  497
Number of correctly predicted samples: 1997/5000 (39.94%)
